<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Demo de Yolo (you only looks once)

Detección de objetos en imagenes<br>

Esta demo está basada en el siguiente repositorio<br>
[Dataset source](https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3)
<br>
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
!pip install tensorflow==2.3.1
!pip install tensorflow-gpu==2.3.1

In [ ]:
# Descargar la carpeta demo
if os.access('yolo_demo', os.F_OK) is False:
    if os.access('yolo_demo.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://github.com/InoveAlumnos/transfer_learning_python/raw/main/anexo/yolo_demo.zip > yolo_demo.zip
        else:
            !wget yolo_demo.zip https://github.com/InoveAlumnos/transfer_learning_python/raw/main/anexo/yolo_demo.zip
    !unzip -q yolo_demo.zip
else:
    print("El archivo ya se encuentra descargado")

In [ ]:
# Descargar los pesos
import gdown
if os.access('./model_data/yolov3.weights', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1xOpcOlyZazI3FG5cnDoMHdE3cQuy_s5Z'
    output = './model_data/yolov3.weights'
    gdown.download(url, output, quiet=False)

In [ ]:
# Descargar la imagen de prueba
if os.access('street.jpg', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/transfer_learning_python/main/imagenes/street.jpg > street.jpg
    else:
        !wget street.jpg https://raw.githubusercontent.com/InoveAlumnos/transfer_learning_python/main/imagenes/street.jpg

In [ ]:
# Descargar el video de prueba
if os.access('video.mp4', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://github.com/InoveAlumnos/transfer_learning_python/raw/main/imagenes/video.mp4 > video.mp4
    else:
        !wget video.mp4 https://github.com/InoveAlumnos/transfer_learning_python/raw/main/imagenes/video.mp4

## COCO Dataset
https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/

## Ensayos

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import tensorflow as tf
import time
from google.colab.patches import cv2_imshow

In [ ]:
from yolov3.utils import detect_image, detect_realtime, detect_video, Load_Yolo_model, detect_video_realtime_mp
from yolov3.configs import *
def detectar_imagen(image_path):
    out_path = "yolo_" + os.path.basename(image_path)
    print("input source:", image_path)
    print("output source:", out_path)

    yolo = Load_Yolo_model()
    new_image = detect_image(yolo, image_path, out_path, input_size=YOLO_INPUT_SIZE, show=False, rectangle_colors=(255,0,0))
    cv2_imshow(new_image)

In [ ]:
detectar_imagen("street.jpg")

In [ ]:
from yolov3.utils import detect_image, detect_realtime, detect_video, Load_Yolo_model, detect_video_realtime_mp
from yolov3.configs import *
def detectar_video(video_path):
    out_path = "yolo_" + os.path.basename(video_path)
    print("input source:", video_path)
    print("output source:", out_path)

    yolo = Load_Yolo_model()
    detect_video(yolo, video_path, out_path, input_size=YOLO_INPUT_SIZE, show=False, rectangle_colors=(255,0,0))

In [ ]:
detectar_video("video.mp4")

In [ ]:
import numpy as np
from yolov3.utils import detect_frame, Load_Yolo_model
from yolov3.configs import *

video_path = "video.mp4"
video_out_path = "yolo2_" + os.path.basename(video_path)
print("input source:", video_path)
print("output source:", video_out_path)
vid = cv2.VideoCapture(video_path)
#vid = cv2.VideoCapture(0) # Para captar una webcam

# Datos del video
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_out_path, codec, fps, (width, height)) # output_path must be .mp4

print(f"Video de {width}x{height} de {fps}fps")

yolo = Load_Yolo_model()

while True:
    _, frame = vid.read()

    if np.shape(frame) == ():
        # Fin del video
        break

    t1 = time.time()

    # Detectar personas en este frame
    new_image = detect_frame(yolo, frame, output_path='', input_size=YOLO_INPUT_SIZE, show=False, rectangle_colors=(255,0,0))

    t2 = time.time()

    fps = 1 / (t2-t1)
      
    print(f"Detection FPS: {fps:.1f}")
    # Grabar detecciones en un archivo
    if video_out_path != '':
        out.write(new_image)

# Cerrar ventanas (no es el caso de colab)
cv2.destroyAllWindows()

# Cerrar archivo de video (file close)
out.release()